In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from pathlib import Path
import os
import sys

In [2]:
inputs = pd.read_csv('inputs.csv')
inputs

,id,eventsoe_no,far_part,acft_make,acft_model,fixed_retractable,acft_category,homebuilt,flight_hours_mean,crew_category,...,crew_sex,certs_held,dprt_apt_id,dest_apt_id,flt_plan_filed,pc_profession,eng_type,carb_fuel_injection,type_fly,eng_mfgr
0,20080107X000261,250,91,PIPER,PA 28-180,FIXD,AIR,N,705.0,PLT,...,M,Y,O69,OQ3,NONE,No,REC,CARB,PERS,Textron Lycoming


In [3]:
import sys

sys.path.insert(0, '../')

In [4]:
from wingman_api.ml_logic.model import initialize_model, fit_model


Loading TensorFlow...


2023-07-05 14:54:10.875411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 14:54:15.630516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-05 14:54:15.630627: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-05 14:54:16.289688: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-05 14:54:28.252825: W tensorflow/stream_executor/platform/de


✅ TensorFlow loaded (25.47s)


In [5]:
model = initialize_model()

✅ Model initialized


In [6]:
sys.path.insert(0, os.getcwd() + '/../wingman_api/ml_logic/')

In [7]:
from wingman_api.ml_logic.preprocessor import preprocess_features

In [8]:
from wingman_api.ml_logic.data import *

In [9]:
from wingman_api.params import *

In [10]:
query = f"""SELECT * FROM {GCP_PROJECT}.{BQ_DATASET}.{BQ_TABLE}"""
query

'SELECT * FROM wingman-project-391115.wingman_dataset.wingman_data_v5'

In [16]:
# ## Import data from GBQ
query = f"""SELECT * FROM {GCP_PROJECT}.{BQ_DATASET}.{BQ_TABLE}"""
data_query_cache_path = Path(LOCAL_DATA_PATH).joinpath("preclean", f"query_{DATA_SIZE}.csv")
if not os.path.exists(Path(LOCAL_DATA_PATH).joinpath("preclean")):
    # Create the directory
    os.makedirs(Path(LOCAL_DATA_PATH).joinpath("preclean"))
wingman_data = get_data_with_cache(GCP_PROJECT, query, data_query_cache_path)


Load data from local CSV...
✅ Data loaded, with shape (6935, 41)


In [17]:
target_columns_v1 = ['phase_no', 'eventsoe_no']
target_columns_v2 = ['category_no', 'subcategory_no', 'section_no', 'subsection_no', 'modifier_no']
target_columns_v3 = ['category_no']
target_columns_v4 = ['eventsoe_no']
target_columns_v5 = ['subcategory_no']

In [18]:
wingman_data

,id,far_part,flt_plan_filed,acft_make,acft_model,total_seats,num_eng,fixed_retractable,cert_max_gr_wt,acft_category,...,available_restraint,eng_no,eng_type,eng_mfgr,eng_model,power_units,hp_or_lbs,carb_fuel_injection,propeller_type,subcategory_no
0,20170821X842081,091,UNK,LUSCOMBE,8,2.0,1.0,FIXD,1350.0,AIR,...,U,1,REC,Continental,A&C65,NaN,NaN,CARB,NaN,6
1,20200106X905571,091,NONE,Glassair,I,2.0,1.0,FIXD,NaN,AIR,...,4,1,REC,Superior,XPO-320-B1AC2,NaN,NaN,CARB,NaN,7
2,20080321X003621,091,NONE,Beech,23,4.0,1.0,FIXD,NaN,AIR,...,NaN,1,REC,Lycoming,O-320,NaN,NaN,NaN,NaN,6
3,20190118X922071,091,NONE,Beech,36,NaN,1.0,RETR,3651.0,AIR,...,U,1,REC,Lycoming,TSIO-540,NaN,NaN,FINJ,CONP,5
4,20120509X232241,091,NONE,BEECH,58,NaN,2.0,FIXD,NaN,AIR,...,NaN,1,REC,Continental,IO540,NaN,NaN,FINJ,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6930,20150908X352411,129,IFR,BOEING COMPANY,BOEING 777-236,297.0,2.0,RETR,632499.0,AIR,...,5,1,TF,GE,GE90-85BG11,85000.0,NaN,NaN,NaN,5
6931,20080912X014381,121,IFR,BOEING,747-209B,8.0,4.0,RETR,679845.0,AIR,...,NaN,1,TF,Pratt & Whitney,JT9D-7Q,51100.0,LBS,NaN,NaN,5
6932,20110621X320401,121,IFR,BOEING,757-222,178.0,2.0,RETR,NaN,AIR,...,NaN,1,TJ,P & W,PW2040,40900.0,LBS,NaN,NaN,1
6933,20150211X146051,121,IFR,EMBRAER,ERJ 190 100 IGW,105.0,2.0,RETR,114100.0,AIR,...,4,1,TF,General Electric,CR3410E6,17390.0,LBS,NaN,NaN,2


In [19]:
# Clean
wingman_data_clean = clean_data(wingman_data)


/home/givenkiban1/lewagon/wingman-backend/notebooks/../wingman_api/ml_logic/data.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wingman_data_cleaned.drop([


In [20]:
wingman_data_clean

,far_part,acft_make,fixed_retractable,acft_category,homebuilt,flight_hours_mean,crew_category,subcategory_no,dprt_time,cert_max_gr_wt,...,crew_sex,certs_held,dprt_apt_id,dest_apt_id,flt_plan_filed,pc_profession,eng_type,carb_fuel_injection,type_fly,eng_mfgr
id,,,,,,,,,,,,,,,,,,,,,
20180529X205531,091,BEECH,RETR,AIR,N,1633.0,PLT,6,1445,2299,...,M,Y,5T6,LRU,NONE,Yes,REC,CARB,PERS,LYCOMING
20130310X027201,091,MAULE,FIXD,AIR,N,946.0,PLT,6,1853,2350,...,M,Y,SMD,SMD,IFR,No,REC,CARB,PERS,LYCOMING
20160627X802361,091,CESSNA,FIXD,AIR,N,4100.0,PLT,6,1730,2350,...,M,Y,BTF,U14,NONE,No,REC,CARB,PERS,CONT MOTOR
20171203X543351,091,CESSNA,FIXD,AIR,N,35.0,DSTU,6,2012,3350,...,M,Y,BAM,BAM,CVFR,Yes,TS,UNK,AOBV,Rolls Royve
20130409X046311,091,CESSNA,FIXD,AIR,N,220.0,PLT,6,30,2450,...,M,Y,2R4,CEU,NONE,Yes,REC,FINJ,BUS,"Continental Motors, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20131005X301191,135,PIPER,FIXD,AIR,N,5573.0,PLT,6,130,9500,...,M,N,TCY,RBL,CVFR,Yes,TS,UNK,POSI,LYCOMING
20141016X331311,091,PIPER,FIXD,AIR,N,190.0,PLT,6,1450,6785,...,M,Y,LRU,AVQ,NONE,UNK,REC,CARB,PERS,Rolls Royce
20080205X001361,091,PIPER,RETR,AIR,N,280.0,PLT,2,2200,17120,...,M,Y,DEN,CYS,IFR,Yes,TP,UNK,FERY,Pratt & Whitney


In [21]:
# Preprocess
wingman_data_proc = preprocess_features(wingman_data_clean, target_columns=target_columns_v5)

/home/givenkiban1/lewagon/wingman-backend/notebooks/../wingman_api/ml_logic/encoders.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wingman_data_enc[''] = np.where(wingman_data_enc['type_fly'].isin(top_9_categories), wingman_data_enc['type_fly'], 'Other')
/home/givenkiban1/lewagon/wingman-backend/notebooks/../wingman_api/ml_logic/encoders.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eng_mfgr['eng_mfgr'] = eng_mfgr['eng_mfgr'].str.upper()
/home/givenkiban1/lewagon/wingman-backend/notebooks/../w

In [22]:
X = wingman_data_proc.drop(columns=["subcategory_no"])
y = wingman_data_proc["subcategory_no"]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [24]:
baseline_mod = RandomForestClassifier(n_estimators=100, random_state=1)
baseline_mod.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [25]:
base = max(y.value_counts()/len(y))
print("Random Selection Accuracy: %.2f%%" % (base * 100.0))

Random Selection Accuracy: 55.51%


In [26]:
import pickle
import time

In [ ]:
filename = "my_model.pickle"

# save model
pickle.dump(baseline_mod, open(filename, "wb"))

In [27]:
from wingman_api.ml_logic.registry import save_model

In [28]:
save_model(baseline_mod)

✅ Model saved locally


In [ ]:
def save_model(model: RandomForestClassifier) -> None:
    """
    Persist trained model locally on the hard drive at f"{LOCAL_REGISTRY_PATH}/models/{timestamp}.h5"
    - if MODEL_TARGET='gcs', also persist it in your bucket on GCS at "models/{timestamp}.h5" --> unit 02 only
    - if MODEL_TARGET='mlflow', also persist it on MLflow instead of GCS (for unit 0703 only) --> unit 03 only
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")

    # Save model locally
    model_path = os.path.join(LOCAL_REGISTRY_PATH, "models", f"{timestamp}.pickle")
    model.save(model_path)

    print("✅ Model saved locally")

    if MODEL_TARGET == "gcs":
        # 🎁 We give you this piece of code as a gift. Please read it carefully! Add a breakpoint if needed!

        model_filename = model_path.split("/")[-1] # e.g. "20230208-161047.h5" for instance
        client = storage.Client()
        bucket = client.bucket(BUCKET_NAME)
        blob = bucket.blob(f"models/{model_filename}")
        blob.upload_from_filename(model_path)

        print("✅ Model saved to GCS")

        return None

    if MODEL_TARGET == "mlflow":
        mlflow.tensorflow.log_model(
            model=model,
            artifact_path="model",
            registered_model_name=MLFLOW_MODEL_NAME
        )

        print("✅ Model saved to MLflow")

        return None

    return None